In [1]:
import pandas as pd
read_file = pd.read_csv (r'C:\\Users\\revan\\Documents\\car_data\car.data')
read_file.to_csv (r'C:\\Users\\revan\\Documents\\car_data\car.csv',index=None)
read_file

,Unnamed: 0,vhigh,vhigh.1,2,2.1,small,low,unacc
0,0,vhigh,vhigh,2,2,small,med,unacc
1,1,vhigh,vhigh,2,2,small,high,unacc
2,2,vhigh,vhigh,2,2,med,low,unacc
3,3,vhigh,vhigh,2,2,med,med,unacc
4,4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...,...
1722,1722,low,low,5more,more,med,med,good
1723,1723,low,low,5more,more,med,high,vgood
1724,1724,low,low,5more,more,big,low,unacc
1725,1725,low,low,5more,more,big,med,good


In [5]:
#important simple linear regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# List of features for simple linear regression
features = car_data_encoded.columns.difference(['class'])
output_features = car_data_encoded.columns

best_input_feature = None
best_output_feature = None
best_accuracy = 0
best_mse_train = float('inf')
best_mse_test = float('inf')

# Iterate over each input feature and output feature
for input_feature in features:
    for output_feature in output_features:
        if output_feature == input_feature:
            continue

        # Step 3: Split the Data into Features and Target
        X = car_data_encoded[[input_feature]]  # Single feature for simple linear regression
        y = car_data_encoded[output_feature]  # Target variable

        # Step 4: Balance the Dataset using SMOTE
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)

        # Step 5: Split the Resampled Data into Training and Testing Sets
        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

        # Step 6: Apply Simple Linear Regression
        linear_model = LinearRegression()
        linear_model.fit(X_train, y_train)

        # Predictions
        y_pred_train = linear_model.predict(X_train)
        y_pred_test = linear_model.predict(X_test)

        # Evaluate the model
        mse_train = mean_squared_error(y_train, y_pred_train)
        mse_test = mean_squared_error(y_test, y_pred_test)

        # Convert continuous predictions to nearest integer class predictions
        y_pred_train_class = np.round(y_pred_train).astype(int)
        y_pred_test_class = np.round(y_pred_test).astype(int)

        # Ensure the predictions are within the range of class labels
        y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
        y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

        # Calculate normalized accuracy
        train_accuracy = accuracy_score(y_train, y_pred_train_class)
        test_accuracy = accuracy_score(y_test, y_pred_test_class)

        # Check if this feature combination gives the best accuracy
        if test_accuracy > best_accuracy:
            best_input_feature = input_feature
            best_output_feature = output_feature
            best_accuracy = test_accuracy
            best_mse_train = mse_train
            best_mse_test = mse_test

        print(f"Input Feature: {input_feature}, Output Feature: {output_feature}")
        print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
        print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")
        print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
        print(f"Test Normalized Accuracy: {test_accuracy:.2f}")
        print("="*30)

print(f"Best Input Feature: {best_input_feature}")
print(f"Best Output Feature: {best_output_feature}")
print(f"Best Train Mean Squared Error (MSE): {best_mse_train:.2f}")
print(f"Best Test Mean Squared Error (MSE): {best_mse_test:.2f}")
print(f"Best Test Normalized Accuracy: {best_accuracy:.2f}")


Input Feature: buying_high, Output Feature: class
Train Mean Squared Error (MSE): 1.20
Test Mean Squared Error (MSE): 1.23
Train Normalized Accuracy: 0.18
Test Normalized Accuracy: 0.19
Input Feature: buying_high, Output Feature: buying_low
Train Mean Squared Error (MSE): 0.20
Test Mean Squared Error (MSE): 0.19
Train Normalized Accuracy: 0.66
Test Normalized Accuracy: 0.69
Input Feature: buying_high, Output Feature: buying_med
Train Mean Squared Error (MSE): 0.20
Test Mean Squared Error (MSE): 0.20
Train Normalized Accuracy: 0.67
Test Normalized Accuracy: 0.66
Input Feature: buying_high, Output Feature: buying_vhigh
Train Mean Squared Error (MSE): 0.20
Test Mean Squared Error (MSE): 0.19
Train Normalized Accuracy: 0.66
Test Normalized Accuracy: 0.67
Input Feature: buying_high, Output Feature: maint_high
Train Mean Squared Error (MSE): 0.25
Test Mean Squared Error (MSE): 0.25
Train Normalized Accuracy: 0.51
Test Normalized Accuracy: 0.48
Input Feature: buying_high, Output Feature: main

logistic regression

In [9]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from itertools import combinations

# Load the dataset (assuming it's stored as a CSV file)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
data = pd.read_csv(url, names=column_names)

# Explore the dataset
print(data.head())

# Preprocessing: Convert categorical variables to numerical using LabelEncoder
label_encoder = LabelEncoder()
for col in data.columns:
    data[col] = label_encoder.fit_transform(data[col])

# Splitting the dataset into features (X) and target variable (y)
y = data['class']

# Standardizing the features (optional but recommended for logistic regression)
scaler = StandardScaler()

# Create a logistic regression model
model = LogisticRegression(max_iter=1000)

# Function to evaluate the model with a combination of features
def evaluate_model(features, y):
    X = data[list(features)]
    # Splitting the data into training and testing sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardizing the features
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Training the model
    model.fit(X_train, y_train)

    # Predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Iterate through all combinations of features to find the best model
best_accuracy = 0
best_features = None

# Generate all combinations of features
feature_combinations = []

# Iterate through all the columns except for the target column 'class'
for i in range(1, len(data.columns)):
    for features in combinations(data.columns[:-1], i):
        feature_combinations.append(features)

for features in feature_combinations:
    accuracy = evaluate_model(features, y)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_features = features

print(f"Best Accuracy: {best_accuracy:.2f}")
print(f"Best Feature Combination: {best_features}")


  buying  maint doors persons lug_boot safety  class
0  vhigh  vhigh     2       2    small    low  unacc
1  vhigh  vhigh     2       2    small    med  unacc
2  vhigh  vhigh     2       2    small   high  unacc
3  vhigh  vhigh     2       2      med    low  unacc
4  vhigh  vhigh     2       2      med    med  unacc
Best Accuracy: 0.70
Best Feature Combination: ('persons', 'safety')


In [1]:
#simple linear regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# For simple linear regression, select one feature
predictor_feature = 'buying_vhigh'  # Change this to any feature you want to use

if predictor_feature not in car_data_encoded.columns:
    predictor_feature = car_data_encoded.columns[0]  # Fallback to first feature if selected feature is not available

# Step 3: Split the Data into Features and Target
X = car_data_encoded[[predictor_feature]]  # Single feature for simple linear regression
y = car_data_encoded['class']  # Target variable

# Step 4: Balance the Dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Split the Resampled Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 6: Apply Simple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Simple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Simple Linear Regression: {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE) for Simple Linear Regression: 1.25
Test Mean Squared Error (MSE) for Simple Linear Regression: 1.22
Train Normalized Accuracy: 0.17
Test Normalized Accuracy: 0.19


In [2]:
#linear regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# For simple linear regression, select one feature
predictor_feature = 'buying_vhigh'  # Change this to any feature you want to use

if predictor_feature not in car_data_encoded.columns:
    predictor_feature = car_data_encoded.columns[0]  # Fallback to first feature if selected feature is not available

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded[[predictor_feature]]  # Single feature for simple linear regression
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Simple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Simple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Simple Linear Regression: {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE) for Simple Linear Regression: 0.75
Test Mean Squared Error (MSE) for Simple Linear Regression: 0.82
Train Normalized Accuracy: 0.71
Test Normalized Accuracy: 0.68


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Multiple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Multiple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Multiple Linear Regression: {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE) for Multiple Linear Regression: 0.54
Test Mean Squared Error (MSE) for Multiple Linear Regression: 0.62
Train Normalized Accuracy: 0.57
Test Normalized Accuracy: 0.54


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Multiple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Multiple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Multiple Linear Regression: {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE) for Multiple Linear Regression: 0.54
Test Mean Squared Error (MSE) for Multiple Linear Regression: 0.62
Train Normalized Accuracy: 0.57
Test Normalized Accuracy: 0.54


In [2]:
#important multiple regression

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Balance the Dataset using SMOTE
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 4: Split the Resampled Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 5: Apply Multiple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Multiple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Multiple Linear Regression: {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE) for Multiple Linear Regression: 1.11
Test Mean Squared Error (MSE) for Multiple Linear Regression: 1.09
Train Normalized Accuracy: 0.38
Test Normalized Accuracy: 0.39


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Simple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Simple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Simple Linear Regression: {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.4

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')




Train Mean Squared Error (MSE) for Simple Linear Regression: 0.54
Test Mean Squared Error (MSE) for Simple Linear Regression: 0.62
Value-based Accuracy on Train Set with threshold 0.4: 50.36%


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Simple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Simple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Simple Linear Regression: {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.7

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')




Train Mean Squared Error (MSE) for Simple Linear Regression: 0.54
Test Mean Squared Error (MSE) for Simple Linear Regression: 0.62
Value-based Accuracy on Train Set with threshold 0.7: 62.30%


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Simple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Simple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Simple Linear Regression: {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.8

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')




Train Mean Squared Error (MSE) for Simple Linear Regression: 0.54
Test Mean Squared Error (MSE) for Simple Linear Regression: 0.62
Value-based Accuracy on Train Set with threshold 0.8: 68.09%


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the Dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# Step 2: Apply One-Hot Encoding to Categorical Variables
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# Step 3: Split the Data into Training and Testing Sets
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Apply Simple Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Simple Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Simple Linear Regression: {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 2

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')




Train Mean Squared Error (MSE) for Simple Linear Regression: 0.54
Test Mean Squared Error (MSE) for Simple Linear Regression: 0.62
Value-based Accuracy on Train Set with threshold 2: 98.41%


multivariable regression

In [8]:
#polynomial regression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# One-hot encode the categorical features
categorical_features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
car_data = pd.get_dummies(car_data, columns=categorical_features)

# Convert the target variable to numerical values
label_encoder = LabelEncoder()
car_data['class'] = label_encoder.fit_transform(car_data['class'])

# Choose the features and the target variable
X = car_data.drop('class', axis=1)  # All features except the target
y = car_data['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply polynomial feature transformation
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predictions
y_pred_train = model.predict(X_train_poly)
y_pred_test = model.predict(X_test_poly)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE): 0.34
Test Mean Squared Error (MSE): 0.45
Train Normalized Accuracy: 0.62
Test Normalized Accuracy: 0.58


In [3]:
#important polynomial regression

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from imblearn.over_sampling import SMOTE

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# One-hot encode the categorical features
categorical_features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
car_data = pd.get_dummies(car_data, columns=categorical_features)

# Convert the target variable to numerical values
label_encoder = LabelEncoder()
car_data['class'] = label_encoder.fit_transform(car_data['class'])

# Choose the features and the target variable
X = car_data.drop('class', axis=1)  # All features except the target
y = car_data['class']  # Target variable

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Apply polynomial feature transformation
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predictions
y_pred_train = model.predict(X_train_poly)
y_pred_test = model.predict(X_test_poly)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")

# Convert continuous predictions to nearest integer class predictions
y_pred_train_class = np.round(y_pred_train).astype(int)
y_pred_test_class = np.round(y_pred_test).astype(int)

# Ensure the predictions are within the range of class labels
y_pred_train_class = np.clip(y_pred_train_class, 0, len(label_encoder.classes_) - 1)
y_pred_test_class = np.clip(y_pred_test_class, 0, len(label_encoder.classes_) - 1)

# Calculate normalized accuracy
train_accuracy = accuracy_score(y_train, y_pred_train_class)
test_accuracy = accuracy_score(y_test, y_pred_test_class)

print(f"Train Normalized Accuracy: {train_accuracy:.2f}")
print(f"Test Normalized Accuracy: {test_accuracy:.2f}")


Train Mean Squared Error (MSE): 0.54
Test Mean Squared Error (MSE): 0.65
Train Normalized Accuracy: 0.63
Test Normalized Accuracy: 0.63


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# One-hot encode the categorical features
categorical_features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
car_data = pd.get_dummies(car_data, columns=categorical_features)

# Convert the target variable to numerical values
label_encoder = LabelEncoder()
car_data['class'] = label_encoder.fit_transform(car_data['class'])

# Choose the features and the target variable
X = car_data.drop('class', axis=1)  # All features except the target
y = car_data['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply polynomial feature transformation
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predictions
y_pred_train = model.predict(X_train_poly)
y_pred_test = model.predict(X_test_poly)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.5

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')



Train Mean Squared Error (MSE): 0.34
Test Mean Squared Error (MSE): 0.45
Value-based Accuracy on Train Set with threshold 0.5: 62.45%


In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# One-hot encode the categorical features
categorical_features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
car_data = pd.get_dummies(car_data, columns=categorical_features)

# Convert the target variable to numerical values
label_encoder = LabelEncoder()
car_data['class'] = label_encoder.fit_transform(car_data['class'])

# Choose the features and the target variable
X = car_data.drop('class', axis=1)  # All features except the target
y = car_data['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply polynomial feature transformation
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predictions
y_pred_train = model.predict(X_train_poly)
y_pred_test = model.predict(X_test_poly)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 3.0

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')



Train Mean Squared Error (MSE): 0.34
Test Mean Squared Error (MSE): 0.45
Value-based Accuracy on Train Set with threshold 3.0: 100.00%


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# One-hot encode the categorical features
categorical_features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
car_data = pd.get_dummies(car_data, columns=categorical_features)

# Convert the target variable to numerical values
label_encoder = LabelEncoder()
car_data['class'] = label_encoder.fit_transform(car_data['class'])

# Choose the features and the target variable
X = car_data.drop('class', axis=1)  # All features except the target
y = car_data['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply polynomial feature transformation
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predictions
y_pred_train = model.predict(X_train_poly)
y_pred_test = model.predict(X_test_poly)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.5

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')



Train Mean Squared Error (MSE): 0.34
Test Mean Squared Error (MSE): 0.45
Value-based Accuracy on Train Set with threshold 0.5: 62.45%


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

# One-hot encode the categorical features
categorical_features = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
car_data = pd.get_dummies(car_data, columns=categorical_features)

# Convert the target variable to numerical values
label_encoder = LabelEncoder()
car_data['class'] = label_encoder.fit_transform(car_data['class'])

# Choose the features and the target variable
X = car_data.drop('class', axis=1)  # All features except the target
y = car_data['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply polynomial feature transformation
poly = PolynomialFeatures(degree=2)  # Change the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predictions
y_pred_train = model.predict(X_train_poly)
y_pred_test = model.predict(X_test_poly)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE): {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE): {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.5

# Calculate value-based accuracy for train set
value_accuracy_train = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Train Set with threshold {threshold}: {value_accuracy_train * 100:.2f}%')



Train Mean Squared Error (MSE): 0.34
Test Mean Squared Error (MSE): 0.45
Value-based Accuracy on Train Set with threshold 0.5: 62.45%


In [18]:
# multivariable regression

In [22]:
import pandas as pd
import numpy as np

# Load the car evaluation dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
car_data = pd.read_csv(url, names=column_names)

print(car_data.head())


  buying  maint doors persons lug_boot safety  class
0  vhigh  vhigh     2       2    small    low  unacc
1  vhigh  vhigh     2       2    small    med  unacc
2  vhigh  vhigh     2       2    small   high  unacc
3  vhigh  vhigh     2       2      med    low  unacc
4  vhigh  vhigh     2       2      med    med  unacc


In [23]:
# One-hot encode the categorical features
car_data_encoded = pd.get_dummies(car_data, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Convert the target variable to numerical values using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
car_data_encoded['class'] = label_encoder.fit_transform(car_data_encoded['class'])

# print(car_data_encoded.head())


In [24]:
from sklearn.model_selection import train_test_split

# Choose the features and the target variable
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
from sklearn.model_selection import train_test_split

# Choose the features and the target variable
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Linear Regression: {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 0.5

# Calculate value-based accuracy for train set
value_accuracy_test = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Test Set with threshold {threshold}: {value_accuracy_test * 100:.2f}%')



Train Mean Squared Error (MSE) for Linear Regression: 0.54
Test Mean Squared Error (MSE) for Linear Regression: 0.62
Value-based Accuracy on Test Set with threshold 0.5: 56.58%


In [32]:
from sklearn.model_selection import train_test_split

# Choose the features and the target variable
X = car_data_encoded.drop('class', axis=1)  # All features except the target
y = car_data_encoded['class']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f"Train Mean Squared Error (MSE) for Linear Regression: {mse_train:.2f}")
print(f"Test Mean Squared Error (MSE) for Linear Regression: {mse_test:.2f}")

# Define a threshold for accuracy
threshold = 3

# Calculate value-based accuracy for train set
value_accuracy_test = np.mean(np.abs(y_train - y_pred_train) <= threshold)
print(f'Value-based Accuracy on Test Set with threshold {threshold}: {value_accuracy_test * 100:.2f}%')



Train Mean Squared Error (MSE) for Linear Regression: 0.54
Test Mean Squared Error (MSE) for Linear Regression: 0.62
Value-based Accuracy on Test Set with threshold 3: 100.00%


In [280]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Reshape X to a column vector
X = x2.reshape(-1, 1)

# Degree of polynomial
degree = 2

# Transform the features to include polynomial terms up to the specified degree
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)

# Initialize and fit a linear regression model
model = LinearRegression()
model.fit(X_poly, y2)



# Predict
y2_pred = model.predict(X_poly)

## Calculate the mean squared error
mse = mean_squared_error(y2, y2_pred)
mse

# # Fit the model
# model = LogisticRegression()
# model.fit(X_poly, y2train)


0.2010053436613673

In [281]:

# # Sample data
# X = np.array([1, 2, 3, 4, 5, 6])
# y = np.array([2, 4, 6, 8, 10, 12])



# # Predict
# y_pred = model.predict(X_poly)


# # Plot the original data and the polynomial regression line
# plt.scatter(X, y, label='Original Data')
# plt.plot(X, y_pred, color='red', label='Polynomial Regression')
# plt.xlabel('X')
# plt.ylabel('y')
# plt.title(f'Polynomial Regression (Degree {degree}), MSE: {mse:.2f}')
# plt.legend()
# plt.show()
read_filen

,Unnamed: 0,vhigh,vhigh.1,2,2.1,small,low,unacc
0,0,4,4,2,2,small,med,0
1,1,4,4,2,2,small,high,0
2,2,4,4,2,2,med,low,0
3,3,4,4,2,2,med,med,0
4,4,4,4,2,2,med,high,0
...,...,...,...,...,...,...,...,...
1722,1722,1,1,5,more,med,med,1
1723,1723,1,1,5,more,med,high,1
1724,1724,1,1,5,more,big,low,0
1725,1725,1,1,5,more,big,med,1


In [283]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Sample data
X4 = read_filen.iloc[:,[1]].values
y4 = read_filen.iloc[:,7].values

# Reshape X to a column vector
X = X4.reshape(-1, 1)

# Initialize and fit a simple linear regression model
model = LinearRegression()
model.fit(X, y4)

# Predict
y4_pred = model.predict(X)

# Calculate evaluation metrics
#simple linear regression
mse = mean_squared_error(y4, y4_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")


Mean Squared Error (MSE): 0.20


In [149]:
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

# # Sample dataset: text strings and corresponding labels
# text_data = [
#     ("This movie is awesome!", 1),
#     ("What an awesome performance!", 1),
#     ("I don't think this is awesome.", 0),
#     ("This is an average movie.", 0),
#     ("The weather is awesome today.", 1)
# ]

# # Splitting data into text and labels
# texts = [data[0] for data in text_data]
# labels = [data[1] for data in text_data]

# # Creating a bag-of-words representation of the text data
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(texts)

# # Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# # Creating and training the logistic regression model
# model = LogisticRegression()
# model.fit(X_train, y_train)

# # Making predictions on the test set
# y_pred = model.predict(X_test)

# # Calculating accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Generate some sample data
np.random.seed(0)
x = read_file.iloc[:,1].values
y = read_file.iloc[:,7].values
X = x.reshape(-1, 1)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Polynomial feature transformation
degree = 2
poly_features = PolynomialFeatures(degree=degree)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Make predictions on the test data
y_pred = model.predict(X_train_poly)


# Define a tolerance level for rounding predictions
tolerance = 0.5  # Adjust as needed



# Calculate the percentage of predictions within the tolerance
accuracy_like_metric = np.mean(np.abs(y_test - y_pred) <= tolerance) 

print(f"Accuracy-like Metric: {accuracy_like_metric:.2f}")


NameError: name 'read_file' is not defined

In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

# Generate some sample data
X = read_file.iloc[:,[0,1,2]].values
y = read_file.iloc[:,7].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the logistic regression model
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)

# Predict probabilities for the test data
y_prob = logistic_regression.predict_proba(X_test)[:, 1]  # Probability of class 1

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_prob)

print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.20048910269838893


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

# Generate some sample data
X =read_file.iloc[:,[1]].values
y = read_file.iloc[:,7].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train.reshape(-1,1), y_train)

# Make probability predictions on the test data
y_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1

# Calculate Mean Squared Error (MSE) for probability predictions
mse = mean_squared_error(y_test, y_prob)
print(f"Mean Squared Error (MSE) for Logistic Regression: {mse}")


Mean Squared Error (MSE) for Logistic Regression: 0.19823452118419221


In [32]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Generate some sample data
np.random.seed(0)
X =read_file.iloc[:,1].values
y = read_file.iloc[:,7].values

X = x.reshape(-1, 1)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make continuous predictions on the test data
y_pred_continuous = model.predict(X_test)

# Define a tolerance level for rounding predictions
tolerance = 0.5  # Adjust as needed

# Round predictions to the nearest integer
y_pred_rounded = np.round(y_pred_continuous)

# Calculate the percentage of predictions within the tolerance
accuracy_like_metric = np.mean(np.abs(y_test - y_pred_rounded) <= tolerance) 

print(f"Accuracy-like Metric: {accuracy_like_metric:.2f}")


Accuracy-like Metric: 0.69


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Load the dataset (assuming it's stored as a CSV file)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
data = pd.read_csv(url, names=column_names)
# Assuming 'evaluation' is the target variable and others are features
target = 'class'
features = [col for col in data.columns if col != target]

# Explore the dataset
print(data.head())

# Preprocessing: Convert categorical variables to numerical using LabelEncoder
label_encoder = LabelEncoder()
for col in data.columns:
    data[col] = label_encoder.fit_transform(data[col])


# Initialize variables to store the best feature and its corresponding R-squared value
best_feature = None
best_r2 = -float('inf')

# Loop through each feature to find the one with the best R-squared value
for feature in features:
    # Reshape feature column to be 2D array
    X = data[[feature]].values
    y = data[target].values
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    print(f'Feature: {feature}, R-squared: {r2}')
    
    # Update the best feature if the current feature is better
    if r2 > best_r2:
        best_r2 = r2
        best_feature = feature

print(f'\nBest Feature: {best_feature}, Best R-squared: {best_r2}')

# Plotting the best feature's regression line
import matplotlib.pyplot as plt

X = data[[best_feature]].values
y = data[target].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.plot(X_test, y_pred, color='red', linewidth=2, label='Predicted')
plt.xlabel(best_feature)
plt.ylabel(target)
plt.title('Simple Linear Regression')
plt.legend()
plt.show()


ValueError: could not convert string to float: 'vhigh'